In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-145703", subscription_id="b968fb36-f06a-4c76-a15f-afab68ae7667", resource_group="aml-quickstarts-145703")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-145703
Azure region: southcentralus
Subscription id: b968fb36-f06a-4c76-a15f-afab68ae7667
Resource group: aml-quickstarts-145703


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
# First, tries to get the compute (if it exists).
cpu_cluster_name = "udacity-compute"
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)

Creating.........
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.policy import MedianStoppingPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.core import ScriptRunConfig
from azureml.core import Environment
import os

# Specify parameter sampler
ps = RandomParameterSampling({
        '--C': choice(0.01, 0.1, 1),
        '--max_iter': choice(25, 50, 100, 150, 200)
})
# Specify a Policy
policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=10)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
# Instead, using a curated environment that includes sciKit-Learn, Pandas...
# It may not include numpy or os dependencias, in that case, clone curated 
# environment and customize it adding new dependencies.
minimal_env = Environment.get(workspace=ws, name="AzureML-Tutorial")

config = ScriptRunConfig(source_directory='./training',
                        script='train.py',
                        compute_target=cpu_cluster,
                        environment=minimal_env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig (run_config=config,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hdr = exp.submit(config=hyperdrive_config)


In [7]:
import joblib
# Get your best run and save the model from that run.
RunDetails(hdr).show()

hdr.get_status()

hdr.wait_for_completion(show_output=True)

best_run = hdr.get_best_run_by_primary_metric()
model = best_run.register_model(model_name = 'udacity_hyper', model_path = 'outputs/model_hyper.pt')

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_fb645cb5-57e2-49e3-945e-5fdc03b02c91
Web View: https://ml.azure.com/runs/HD_fb645cb5-57e2-49e3-945e-5fdc03b02c91?wsid=/subscriptions/b968fb36-f06a-4c76-a15f-afab68ae7667/resourcegroups/aml-quickstarts-145703/workspaces/quick-starts-ws-145703&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-05-26T04:56:45.634925][API][INFO]Experiment created<END>\n""<START>[2021-05-26T04:56:46.167527][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-05-26T04:56:46.565369][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-05-26T04:57:16.0042314Z][SCHEDULER][INFO]Scheduling job, id='HD_fb645cb5-57e2-49e3-945e-5fdc03b02c91_3'<END><START>[2021-05-26T04:57:15.9837931Z][SCHEDULER][INFO]Scheduling job, id='HD_fb645cb5-57e2-49e3-945e-5fdc03b02c91_0'<END><START>[2021-05-26T04:57:15.9850390Z][SCHEDULER][INFO]Scheduling job, id='HD_fb645cb5-57e2-4

AttributeError: 'NoneType' object has no attribute 'register_model'

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

datastore_path = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds = Dataset.Tabular.from_delimited_files(path=datastore_path)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

whole_data = pd.concat([x,y], axis = 1)

train_data, test_data = train_test_split(whole_data, test_size=0.2, random_state=40)

train_data.to_csv('training/training_df.csv')

ds_processed = ws.get_default_datastore()
ds_processed.upload(src_dir='training', target_path='train_data')

train_ds = Dataset.Tabular.from_delimited_files(path=[(ds_processed, ('training/training_df.csv'))])


In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_settings = {
    "n_cross_validations": 5,
    "primary_metric": 'accuracy',
    "enable_early_stopping": True,
    "enable_onx_compatible_models": True,
    "experiment_timeout_minutes": 30,
    "verbosity": logging.INFO,
}
# Regression (r2_score) or classification(accuracy).
automl_config = AutoMLConfig(
    task='classification',
    compute_target=cpu_cluster,
    training_data=train_ds,
    label_column_name='y',
    **automl_settings)

In [ ]:
# Submit your automl run

remote_run = exp.submit(automl_config, show_output=False)

In [ ]:
# Retrieve and save your best automl model.

best_run = remote_run.get_best_run_by_primary_metric()
best_run.get_tags()
model = best_run.register_model(model_name = 'udacity_automl', model_path = 'outputs/model_automl.pt')